In [1]:
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function

from collections import OrderedDict
from itertools import product
from os.path import *
from os import *
import pickle
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
from icecube.dataclasses import *
from icecube import *
from icecube.icetray import *
from icecube.clsim import MakeIceCubeMediumProperties

In [4]:
!pwd

/home/justin/src/retro/notebooks


In [5]:
sys.path.append(expandvars('$I3_SRC/clsim/python/util'))
sys.path.append(abspath('..'))

from GetIceTiltZShift import GetIceTiltZShift, I3CLSimScalarFieldIceTiltZShift
from retro.const import get_sd_idx
from retro.utils.geom import powerspace

In [6]:
# Spice lea tables, for use with MC used for GRECO and DRAGON samples; used up until 2019-04-xx
# gcd_path = '../data/GeoCalibDetectorStatus_IC86.55697_corrected_V2.pkl'
# ice_model = 'spice_lea'

# Spice 3.2.1 tables, for use with oscNext sim
gcd_path = '../data/GeoCalibDetectorStatus_AVG_55697-57531_PASS2_SPE_withScaledNoise.pkl'
ice_model = 'spice_3.2.1'

In [7]:
gcd = pickle.load(file(gcd_path, 'rb'))
geo = gcd['geo']
gcd_basename = splitext(basename(gcd_path))[0]
print(gcd_basename)

GeoCalibDetectorStatus_AVG_55697-57531_PASS2_SPE_withScaledNoise


In [31]:
gcd.keys()

['source_gcd_name',
 'source_gcd_md5',
 'source_gcd_i3_md5',
 'geo',
 'noise',
 'rde']

In [23]:
valid_mask = np.logical_not(
    np.logical_not(np.isfinite(gcd['rde'])) | (gcd['rde'] == 0)
    &
    np.logical_not(np.isfinite(gcd['noise'])) | (gcd['noise'] == 0)
)
n_nz = np.count_nonzero(valid_mask)
n_z = np.count_nonzero(valid_mask == 0)
print("{} valid, {} invalid DOMs".format(n_nz, n_z), valid_mask.shape)

5081 valid, 79 invalid DOMs (86, 60)


See `ice_layer_properties_validation.ipynb` for validation that I'm computing layer indices using the CLSim tool correctly.

In [8]:
I3_SRC = os.environ["I3_SRC"]
!cat $I3_SRC/ice-models/resources/models/$ice_model/cfg.txt

# ppc configuration file: follow strict order below
5     # over-R: DOM radius "oversize" scaling factor
0.94   # overall DOM efficiency correction
0.35  # 0=HG; 1=SAM
0.9   # g=<cos(theta)>

130    # direction of major anisotropy axis
-0.069 # magnitude of major anisotropy coefficient k1
0.0345 # magnitude of minor anisotropy coefficient k2


In [9]:
ice_props, ice_params = MakeIceCubeMediumProperties(
    detectorCenterDepth=1948.07*I3Units.m,
    iceDataDirectory=join(I3_SRC, 'ice-models/resources/models/' + ice_model),
    useTiltIfAvailable=True,
    returnParameters=True,
)
ice_params

{'anisotropyDirAzimuth': 2.2689280275926285,
 'anisotropyMagnitudeAlongDir': -0.069,
 'anisotropyMagnitudePerpToDir': 0.0345}

In [10]:
rootdir = '{}/ice-models/resources/models/{}'.format(I3_SRC, ice_model)

cfg = np.loadtxt(join(rootdir, 'cfg.txt'))
print('cfg.txt shape:', cfg.shape)

oversize_scaling = cfg[0]
efficiency_correction = cfg[1]
liu_scattering_fraction = cfg[2]
mean_cos_scattering_theta = cfg[3]

print('mean_cos_scattering_theta:', mean_cos_scattering_theta)

icemodel_par = np.loadtxt(join(rootdir, 'icemodel.par'))
print('icemodel.par shape:', icemodel_par.shape)
if False: #ice_model == 'spice_mie':
    assert icemodel_par.shape[0] == 6
    alpha, kappa, A, B, D, E = icemodel_par[:, 0].T
    print('alpha={}, kappa={}, A={}, B={}, D={}, E={}'.format(alpha, kappa, A, B, D, E))
else: # ice_model == 'spice_lea':
    #assert icemodel_par.shape[0] == 4
    alpha, kappa, A, B = icemodel_par[:, 0].T[:4]
    print('alpha={}, kappa={}, A={}, B={}'.format(alpha, kappa, A, B))
#else:
#    raise ValueError(ice_model)
    
icemodel = np.loadtxt(join(rootdir, 'icemodel.dat'))
print('icemodel.dat shape:', icemodel.shape)

# first column: z layer *depth* midpoints
zlayermidpts = icemodel[:, 0]
# second column: b_e(400) values
be400 = icemodel[:, 1]
# third column: adust(400) values
adust400 = icemodel[:, 2]
# fourth column: delta_tau values
delta_tau = icemodel[:, 3]

cfg.txt shape: (7,)
mean_cos_scattering_theta: 0.9
icemodel.par shape: (4, 2)
alpha=0.898608505726, kappa=1.08410680294, A=6954.09033203, B=6617.75439453
icemodel.dat shape: (171, 4)


In [40]:
%%time

wavelen = 555e-9 * I3Units.m
n_strings = 86
n_doms_per_string = 60
n_doms = 86 * 60
z0 = ice_props.GetLayersZStart()
dz = ice_props.GetLayersHeight()
zshift = ice_props.GetIceTiltZShift()

coords = OrderedDict(
    [
        ("r", powerspace(0.1, 200, 5, 5)),
        ("costheta", np.linspace(np.cos(np.pi), np.cos(0), 9)),
        ("phi", np.arange(0, 2*np.pi, np.pi/4)),
    ]
)
properties = OrderedDict(
    [
        ("ice_model", ice_model),
        ("gcd", gcd),
        ("wavelen", wavelen),
        ("coords", coords),
        ("dom_info", OrderedDict()),
    ]
)
for s_idx, d_idx in product(range(n_strings), range(n_doms_per_string)):
    #if not valid_mask[s_idx, d_idx]:
    #    continue
    string = s_idx + 1
    dom = d_idx + 1
    sd_idx = get_sd_idx(string=string, dom=dom)
    x, y, z = geo[s_idx, d_idx]
    al = []
    sl = []
    for r, costheta, phi in product(coords["r"], coords["costheta"], coords["phi"]):
        sintheta = np.sin(np.arccos(costheta))
        z_ = z + r * costheta
        x_ = x + r * sintheta * np.cos(phi)
        y_ = y + r * sintheta * np.sin(phi)

        # Original sign convention: z + zshift - z0; this has now been
        # verified by comparing against a "manual" calculation of the
        # ice scattering length.
        layer_num = int((z_ - z0 + zshift.GetValue(x_, y_, z_)) / dz)
        # Alternative sign convention
        #layer_num = int((z_ - z0 - zshift.GetValue(x_, y_, z_)) / dz)

        al.append(ice_props.GetAbsorptionLength(layer_num).GetValue(wavelen))
        sl.append(ice_props.GetScatteringLength(layer_num).GetValue(wavelen))

    zequiv = z + zshift.GetValue(x, y, z)
    layer_num = int((zequiv - z0) / dz)
    properties["dom_info"][(string, dom)] = OrderedDict(
        [
            ("x", x),
            ("y", y),
            ("z", z),
            ("zequiv", zequiv),
            ("layer_num", layer_num),
            # functions from icecube_geometry.ipynb for getting "hex" coordinates along 3 hex-aligned axes;
            # not currently being used for any subsequent computations, so commenting "h{0,1,2}" out
            #get_hc0_coord(x, y),
            #get_hc1_coord(x, y),
            #get_hc2_coord(x, y),
            ("absorption_length_at_dom", ice_props.GetAbsorptionLength(layer_num).GetValue(wavelen)),
            ("scattering_length_at_dom", ice_props.GetScatteringLength(layer_num).GetValue(wavelen)),
            ("absorption_length_nearby", np.array(al)),
            ("scattering_length_nearby", np.array(sl)),
        ]
    )

CPU times: user 19.6 s, sys: 488 ms, total: 20.1 s
Wall time: 19.6 s


In [41]:
outfpath = "../data/dom_clustering_properties_binned__{}__{}.pkl".format(ice_model, gcd_basename)
pickle.dump(properties, file(outfpath, "w"), protocol=pickle.HIGHEST_PROTOCOL)
print(outfpath)

../data/dom_clustering_properties_binned__spice_3.2.1__GeoCalibDetectorStatus_AVG_55697-57531_PASS2_SPE_withScaledNoise.pkl
